# Projet - Atelier Data Science

## Introduction

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

### Importation des librairies 

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



# Préparation des données

Nous faisons le choix d'appliquer la préparation des données sur les années 2018 et 2022. En effet, nous pourrons donc voir l'impact du COVID sur les DVF.

### Importation des données

In [3]:
df_2022 = pd.read_csv("./valeursfoncieres-2022.txt", sep="|", decimal=",")
# df_2018 = pd.read_csv("./valeursfoncieres-2018.txt", sep="|", decimal=",")

/var/folders/bk/7b_bwg6x6gjc8hq_4skf_9c00000gn/T/ipykernel_43870/3164777433.py:1: DtypeWarning: Columns (18,23,24,26,28,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2022 = pd.read_csv("./valeursfoncieres-2022.txt", sep="|", decimal=",")


In [4]:
print(df_2022.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3803885 entries, 0 to 3803884
Data columns (total 43 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Identifiant de document     float64
 1   Reference document          float64
 2   1 Articles CGI              float64
 3   2 Articles CGI              float64
 4   3 Articles CGI              float64
 5   4 Articles CGI              float64
 6   5 Articles CGI              float64
 7   No disposition              int64  
 8   Date mutation               object 
 9   Nature mutation             object 
 10  Valeur fonciere             float64
 11  No voie                     float64
 12  B/T/Q                       object 
 13  Type de voie                object 
 14  Code voie                   object 
 15  Voie                        object 
 16  Code postal                 float64
 17  Commune                     object 
 18  Code departement            object 
 19  Code commune         

Nous allons chercher le pourcentage de données manquantes par colonnes pour savoir quelles données nous devons garder.

In [5]:
(df_2022.isnull().sum()/len(df_2022))*100

Identifiant de document       100.000000
Reference document            100.000000
1 Articles CGI                100.000000
2 Articles CGI                100.000000
3 Articles CGI                100.000000
4 Articles CGI                100.000000
5 Articles CGI                100.000000
No disposition                  0.000000
Date mutation                   0.000000
Nature mutation                 0.000000
Valeur fonciere                 0.649888
No voie                        34.733621
B/T/Q                          95.374019
Type de voie                   36.984294
Code voie                       1.026398
Voie                            1.029290
Code postal                     1.029816
Commune                         0.000000
Code departement                0.000000
Code commune                    0.000000
Prefixe de section             95.824295
Section                         0.004075
No plan                         0.000000
No Volume                      99.765792
1er lot         

In [7]:
print("En 2022, 1% correspond à ", int(len(df_2022)*0.01), " valeurs du dataset")
# print("En 2018, 1% correspond à ", int(len(df_2018)*0.01), " valeurs du dataset")


En 2022, 1% correspond à  38038  valeurs du dataset


On supprime les colonnes avec trop de données manquantes et celles qui ne serviront à rien dans le modèle de marchine learning et la data viz.

In [8]:
df_2022.drop(['Code type local', 'Identifiant de document', 'Reference document', '1 Articles CGI', '2 Articles CGI','3 Articles CGI','4 Articles CGI','5 Articles CGI', 'Prefixe de section', 'Identifiant local', 'Nature culture speciale', 'No voie', 'B/T/Q', 'No Volume', '1er lot', 'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot', '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot', 'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot'], axis=1, inplace=True)

On se rend compte que différentes colonnes ont un pourcentage élevée de données manquantes. On fait donc le choix de supprimer les lignes dont une des valeurs parmis ces colonnes est manquante. 

In [9]:
df_2022.dropna(subset=['Type de voie'], inplace=True)
df_2022.dropna(subset=['Type local'], inplace=True)
df_2022.dropna(subset=['Surface reelle bati'], inplace=True)
df_2022.dropna(subset=['Valeur fonciere'], inplace=True)
df_2022.dropna(subset=['Code postal'], inplace=True)

On vérifie ensuite le pourcentage de données manquantes par colonne. On remarque qu'il est maintenant de 0% sauf pour deux colonnes.

In [10]:
(df_2022.isnull().sum()/len(df_2022))*100

No disposition                0.000000
Date mutation                 0.000000
Nature mutation               0.000000
Valeur fonciere               0.000000
Type de voie                  0.000000
Code voie                     0.000000
Voie                          0.000000
Code postal                   0.000000
Commune                       0.000000
Code departement              0.000000
Code commune                  0.000000
Section                       0.005290
No plan                       0.000000
Nombre de lots                0.000000
Type local                    0.000000
Surface reelle bati           0.000000
Nombre pieces principales     0.000000
Nature culture               50.317449
Surface terrain              50.317449
dtype: float64

On remarque que la colonne 'Surface terrain' a environ 50% de valeurs manquantes. Cependant, il ne s'agit pas de réelle valeur manquante. Dans les faits, il s'agit des biens immobiliers qui n'ont pas de terrain. On remplace donc toutes les valeurs manquantes par 0. En effet, les biens sans terrain ont donc un terrain de 0 mètre carré.

In [11]:
df_2022['Surface terrain'] = df_2022['Surface terrain'].fillna(0)

On remarque que certains bien immobiliers composés de plusieurs types de bien (exemple : dépendance et appartement) sont affichés plusieurs fois. Pour éviter que cela fausse les résultats des algorithmes de prédictions, nous allons donc grouper ces lignes. 

In [ ]:
df_2022 = df_2022.groupby(['No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere', 'Type de voie', 'Code voie', 'Voie', 'Commune', 'Code departement', 'Code commune', 'Section', 'Nature culture', 'Nombre de lots', 'Surface terrain'], as_index=False).agg({
    'No disposition': 'first',
    'Date mutation': 'first',
    'Nature mutation': 'first',
    'Valeur fonciere': 'first',
    'Type de voie': 'first',
    'Code postal': 'first',
    'Code departement': 'first',
    'Code commune': 'first',
    'Section': 'first',
    'No plan': 'first',
    'Nombre de lots': 'first',
    'Type local': lambda x: 'Multiple' if len(set(x)) > 1 else x.unique()[0],
    'Surface reelle bati': 'sum',
    'Nombre pieces principales': 'sum',
    'Nature culture': 'first',
    'Surface terrain': 'first'
})

On remarque dans la documentation que les biens ayant plusieurs nature de culture s'affiche en plusieurs ligne, on va donc aussi les grouper.

In [13]:
df_2022 = df_2022.groupby(['No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere', 'Type de voie', 'Voie', 'Commune', 'Code departement', 'Code commune', 'Section', 'Nombre de lots', 'Surface terrain'], as_index=False).agg({
    'No disposition': 'first',
    'Date mutation': 'first',
    'Nature mutation': 'first',
    'Valeur fonciere': 'first',
    'Type de voie': 'first',
    'Code postal': 'first',
    'Code departement': 'first',
    'Code commune': 'first',
    'Section': 'first',
    'No plan': 'first',
    'Nombre de lots': 'first',
    'Type local': 'first',
    'Surface reelle bati': 'first',
    'Nombre pieces principales': 'first',
    'Nature culture': lambda x: 'Multiple' if len(set(x)) > 1 else x.unique()[0],
    'Surface terrain': 'first'
})

On vérifie donc le taux de valeurs manquantes, il n'y en a plus. 

In [14]:
(df_2022.isnull().sum()/len(df_2022))*100

Voie                         0.0
Commune                      0.0
No disposition               0.0
Date mutation                0.0
Nature mutation              0.0
Valeur fonciere              0.0
Type de voie                 0.0
Code postal                  0.0
Code departement             0.0
Code commune                 0.0
Section                      0.0
No plan                      0.0
Nombre de lots               0.0
Type local                   0.0
Surface reelle bati          0.0
Nombre pieces principales    0.0
Nature culture               0.0
Surface terrain              0.0
dtype: float64

In [19]:
train_data, test_data = train_test_split(df_2022, test_size=0.2, random_state=42)

print("Taille de l'ensemble d'apprentissage :", len(train_data))
print("Taille de l'ensemble de test :", len(test_data))


Taille de l'ensemble d'apprentissage : 446253
Taille de l'ensemble de test : 111564


# Analyse Exploratoire des Données

ijdqjodiaz

# Modèle Machine Learning

### A - Apprentissage supervisé du prix des appartements

On commencer par affiner notre dataset aux seuls appartement de la région parisienne. 

In [71]:
colonnes_a_supprimer = ['No disposition', 'Code postal', 'Code commune', 'No plan', 'Voie', 'Commune', 'Nature mutation', 'Type de voie', 'Code departement', 'Section', 'Type local', 'Nature culture', 'Date mutation']

data_paris_2022 = df_2022.drop(colonnes_a_supprimer, axis=1)


In [73]:
train_data, test_data = train_test_split(data_paris_2022, test_size=0.2, random_state=42)

In [74]:
X_train = train_data.drop('Valeur fonciere', axis=1)
y_train = train_data['Valeur fonciere']

X_test = test_data.drop('Valeur fonciere', axis=1)
y_test = test_data['Valeur fonciere']

In [75]:
linear_reg = LinearRegression()
decision_tree_reg = DecisionTreeRegressor()
random_forest_reg = RandomForestRegressor()

In [76]:
linear_reg.fit(X_train, y_train)
decision_tree_reg.fit(X_train, y_train)
random_forest_reg.fit(X_train, y_train)

RandomForestRegressor()

In [77]:
linear_reg_preds = linear_reg.predict(X_test)
decision_tree_preds = decision_tree_reg.predict(X_test)
random_forest_preds = random_forest_reg.predict(X_test)

In [78]:
linear_reg_rmse = mean_squared_error(y_test, linear_reg_preds, squared=False)
decision_tree_rmse = mean_squared_error(y_test, decision_tree_preds, squared=False)
random_forest_rmse = mean_squared_error(y_test, random_forest_preds, squared=False)

linear_reg_mae = mean_absolute_error(y_test, linear_reg_preds)
decision_tree_mae = mean_absolute_error(y_test, decision_tree_preds)
random_forest_mae = mean_absolute_error(y_test, random_forest_preds)

linear_reg_r2 = r2_score(y_test, linear_reg_preds)
decision_tree_r2 = r2_score(y_test, decision_tree_preds)
random_forest_r2 = r2_score(y_test, random_forest_preds)

print("Linear Regression RMSE:", linear_reg_rmse)
print("Decision Tree RMSE:", decision_tree_rmse)
print("Random Forest RMSE:", random_forest_rmse)

print("Linear Regression MAE:", linear_reg_mae)
print("Decision Tree MAE:", decision_tree_mae)
print("Random Forest MAE:", random_forest_mae)

print("Linear Regression R^2:", linear_reg_r2)
print("Decision Tree R^2:", decision_tree_r2)
print("Random Forest R^2:", random_forest_r2)


Linear Regression RMSE: 4090657.4892193987
Decision Tree RMSE: 4530272.867866043
Random Forest RMSE: 4139455.135899413
Linear Regression MAE: 336370.7107420583
Decision Tree MAE: 403813.26807866496
Random Forest MAE: 361029.55077989906
Linear Regression R^2: 0.033262035900212705
Decision Tree R^2: -0.18569028444123026
Random Forest R^2: 0.010059941188250265
